In [9]:
import numpy as np
import pandas as pd

from scipy import stats, special, spatial

import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib.lines import Line2D
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300

import importlib.util
import os

import warnings
warnings.filterwarnings('ignore')

#s.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import keras
import numpy as np
import pandas as pd
import importlib.util
import os

from scipy import stats, special, spatial
from scipy.stats import gaussian_kde

import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib.lines import Line2D
import matplotlib as mpl

# https://github.com/jnothman/UpSetPlot
import upsetplot

import warnings
warnings.filterwarnings('ignore')

#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"


In [24]:
#Datasets
#Ascomycota Native
#AscomycotaXvalCombined = np.load('D:/Users/ernes/Documents/1KCL/Zelezniak/DeepExpressionWorking/2018_11_26/data_combined_df_output/Ascomycota.Xval.npy', allow_pickle=True)
#AscomycotaXhotCombined = np.load('D:/Users/ernes/Documents/1KCL/Zelezniak/DeepExpressionWorking/2018_11_26/data_combined_df_output/Ascomycota.Xhot.npy', allow_pickle=True)
#AscomycotaTestCombined = [AscomycotaXvalCombined, AscomycotaXhotCombined]
#AscomycotaTestCombined = [AscomycotaXhotCombined, AscomycotaXvalCombined]

#Ascomycota Input Data GFP Replaced
GFPAscomycotaXvalCombined = np.load('D:/Users/ernes/Documents/1KCL/Zelezniak/DeepExpressionWorking/2018_11_26/gfp_data_combined_output/Ascomycota_GFP.Xval.npy', allow_pickle=True)
GFPAscomycotaXhotCombined = np.load('D:/Users/ernes/Documents/1KCL/Zelezniak/DeepExpressionWorking/2018_11_26/gfp_data_combined_output/Ascomycota_GFP.Xhot.npy', allow_pickle=True)
#GFPAscomycotaTestCombined = [GFPAscomycotaXvalCombined, GFPAscomycotaXhotCombined]
GFPAscomycotaTestCombined = [GFPAscomycotaXhotCombined, GFPAscomycotaXvalCombined]

#Saccharmoyces
# Data
fn1 = 'scerevisiae_tpm_full_cut5.csv'
fn2 = 'scerevisiae_accessions.tsv'
fn3 = 'scerevisiae.rsd1.lmbda_22.npz'
fn4 = 'Saccharomyces_cerevisiae.R64-1-1.regions.csv'

fn5 = 'organisms_deep_results.csv'

## datasets
folder = 'D:/Users/ernes/Documents/1KCL/Zelezniak/Data1/'

#Model
fname_data1 = folder+'scerevisiae.rsd1.lmbda_22.npz'
fname_module1 = folder+'Model_C3F2all.py'
fname_p1 = folder+'scerevisiae_good_rsd1_lmbda22_Model_C3F2all_1234_c008090fd8e4b9971fe71eed502c9f7b_best.p'
fname_weights1 = folder+'scerevisiae_good_rsd1_lmbda22_Model_C3F2all_1234_c008090fd8e4b9971fe71eed502c9f7b_best'

fname_module2 = folder+'Model_L2_C3F2addvarsopen_loadox.py'
fname_data2 = folder+'scerevisiae.rsd1.lmbda_22.1000.npz'
fname_p2 = folder+'scerevisiae_rsd1_merged_Model_C3F2_open_1241_0c6919106ab536108b55fd92965886b3_best.p'
fname_weights2 = folder+'scerevisiae_rsd1_merged_Model_C3F2_open_1241_0c6919106ab536108b55fd92965886b3_best'

In [11]:
def load_data(fname):
    #X is multi-variable array
    #Y contains single variable - fix shape for Keras

    npzfile = np.load(fname,allow_pickle=True)
    Xh_train = npzfile['arr_0']
    Xh_test = npzfile['arr_1']
    Xv_train = npzfile['arr_2'][:,8:]
    Xv_test = npzfile['arr_3'][:,8:]
    Y_train = npzfile['arr_4']
    Y_test = npzfile['arr_5']
    names_train = npzfile['arr_6']
    names_test = npzfile['arr_7']

    X_train = list()
    X_train.append(Xh_train)
    #X_train.append(Xv_train)
    X_test = list()
    X_test.append(Xh_test)
    #X_test.append(Xv_test)

    Y_train = Y_train.astype(np.float32).reshape((-1,))
    Y_test = Y_test.astype(np.float32).reshape((-1,))

    return (np.concatenate((Y_train, Y_test)), 
            np.concatenate((names_train,names_test)), 
            np.concatenate((Xv_train,Xv_test)))

In [12]:
#load modules - POC model
def load_module(model_path):
    '''loads module containing models given path'''
    spec = importlib.util.spec_from_file_location('module',model_path)
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    return module

In [13]:
#Load data and model
module = load_module(fname_module1)

#Load p
p = pd.read_csv(fname_p1,sep=';')
p = p.iloc[0]
p.drop(['path','overfit'],inplace=True)
print(p.shape)

#Load data
X_train, X_test, Y_train, Y_test = module.load_data(fname_data1) 
# Y = np.concatenate((Y_train, Y_test))
# X = np.concatenate((X_train[0], X_test[0]))
print(len(X_test))
print(len(Y_test))

#Model
input_shapes = [sl.shape[1:] for sl in X_train]
model = module.POC_model(input_shapes, p)
#Set weights model 1
print("Loading model 1 from disk..")
model.load_weights(fname_weights1)

(42,)
2
425
Loading model 1 from disk..


In [15]:
#Predict
#GFP_Ascomycota_pred = model.predict(GFPAscomycotaTestCombined)
#DUE TO MEMORY CONSTRAINTS

# Splitting the data
GFPAscomycotaTestCombined_split1 = [data[:data.shape[0]//2] for data in GFPAscomycotaTestCombined]
GFPAscomycotaTestCombined_split2 = [data[data.shape[0]//2:] for data in GFPAscomycotaTestCombined]

# Predicting on each half
GFPAscomycota_pred1 = model.predict(GFPAscomycotaTestCombined_split1)
GFPAscomycota_pred2 = model.predict(GFPAscomycotaTestCombined_split2)

# Concatenating the two halves
GFP_Ascomycota_pred = np.concatenate((GFPAscomycota_pred1, GFPAscomycota_pred2), axis=0)


2627/2627 [==============================] - 489s 186ms/step


In [17]:
# Concatenating the two halves
GFP_Ascomycota_pred = np.concatenate((GFPAscomycota_pred1, GFPAscomycota_pred2), axis=0)

In [18]:
import numpy as np
#Define the path
file_path = r"D:/Users/ernes/Documents/1KCL/Zelezniak/DeepExpressionWorking/2018_11_26/gfp_data_combined_output/GFP_Ascomycota_pred.npy"
#Save the array to the specified file
np.save(file_path, GFP_Ascomycota_pred)

In [20]:
# Assuming GFPAscomycotaTestCombined is a list of numpy arrays
print([data.shape for data in GFPAscomycotaTestCombined])
# Printing shapes
print([data.shape for data in GFPAscomycotaTestCombined_split1])
print([data.shape for data in GFPAscomycotaTestCombined_split2])

[(168066, 2150, 4), (168066, 72)]
[(84033, 2150, 4), (84033, 72)]
[(84033, 2150, 4), (84033, 72)]


In [ ]:
GFPAscomycotaXvalCombined = np.load('D:/Users/ernes/Documents/1KCL/Zelezniak/DeepExpressionWorking/2018_11_26/gfp_data_combined_output/Ascomycota_GFP.Xval.npy', allow_pickle=True)
GFPAscomycotaXhotCombined = np.load('D:/Users/ernes/Documents/1KCL/Zelezniak/DeepExpressionWorking/2018_11_26/gfp_data_combined_output/Ascomycota_GFP.Xhot.npy', allow_pickle=True)